# Experimentos

Experimentos são a principal forma de agrupar 3 entidades:

1. **Datasets**: conjunto de textos acompanhados de um *target*, representando um problema de classificação ou regressão;
2. **Pipelines**: são capazes de receber um ou mais textos como entrada e produzir uma saída para o problema de classificação/regressão;
3. **Métricas**: permitem avaliar o desempenho de uma ou mais pipelines para o problema em questão;

A biblioteca `nlpbox` disponibiliza diferentes métodos para construção e execução de experimentos em seu pacote `nlpbox.experiments`. É possível construir um experimento instanciando cada um dos componentes individualmente ou através do pacote `nlpbox.factory`, que possui facilidades para obter as diversas classes da biblioteca.

In [1]:
import json

from nlpbox.factory.experiment import SimpleExperimentBuilder

In [2]:
# === Construindo um experimento para classificação no Essay-BR ===
# Por simplicidade, vamos instanciar um experimento
#   para comparar algumas abordagens para classificação
#   da competência 1 do dataset Essay-BR.
builder = SimpleExperimentBuilder()

# Inicialmente, vamos definir o dataset
builder.dataset('essayBR',
                extended=False,
                target_competence='C1')

# Vamos definir o tipo do problema
builder.classification()

# Vamos definir a seed randômica
builder.seed(42)

# Depois, vamos definir algumas métricas
#   que devem ser calculadas
builder.add_metric('precision', average='weighted')
builder.add_metric('recall', average='weighted')
builder.add_metric('f1', average='weighted')
builder.add_metric('kappa')
builder.add_metric('neighborKappa')

# Depois, vamos definir qual a métrica
#   que deve ser utilizar para escolher a
#   melhor pipeline
builder.best_criteria('precision', average='weighted')

# Agora, vamos adicionar algumas pipelines baseadas
#   em extração de característica
builder.add_feature_pipeline(
    features=['readabilityBR',
              'regencyBR',
              'syntacticComplexityBR',
              'textualSimplicityBR'],
    estimators=['svm',
                'etreesClf',
                'lgbmClf',
                'xgbClf'],
    names=['svm+features',
           'etrees+features',
           'lgbm+features',
           'xgb+features'])

# Agora, vamos adicionar algumas pipelines baseadas
#   em outras estratégias de vetorização
builder.add_vectorizer_pipeline('tfidfVectorizer',
                                estimators=['xgbClf'],
                                names=['xgb+tfidf'])
builder.add_feature_pipeline('bertVectorizer',
                             estimators=['etreesClf'],
                             names=['etrees+bert'])

# Uma vez que tenhamos configurado o experimento,
#   podemos obter uma instância:
experiment = builder.build()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [3]:
# === Executando o experimento ===
result = experiment.run()

INFO:nlpbox.experiments.simple_experiment:Setting up experiment...
INFO:nlpbox.experiments.simple_experiment:Obtaining train and test split...
INFO:nlpbox.experiments.simple_experiment:Train has 3656 samples, Test has 914 samples.
INFO:nlpbox.experiments.simple_experiment:Run started.
INFO:nlpbox.experiments.simple_experiment:Started pipeline "svm+features" (1/6)


Vetorização:   0%|          | 0/3656 [00:00<?, ?it/s]

Vetorização:   0%|          | 0/914 [00:00<?, ?it/s]

INFO:nlpbox.experiments.simple_experiment:Started pipeline "etrees+features" (2/6)


Vetorização:   0%|          | 0/3656 [00:00<?, ?it/s]

...

Vetorização:   0%|          | 0/914 [00:00<?, ?it/s]

INFO:nlpbox.experiments.simple_experiment:Started pipeline "etrees+bert" (6/6)


Vetorização:   0%|          | 0/3656 [00:00<?, ?it/s]

Vetorização:   0%|          | 0/914 [00:00<?, ?it/s]

INFO:nlpbox.experiments.simple_experiment:Run finished in 1283.30 seconds.
Best pipeline: etrees+bert


In [8]:
# === Inspecionando os resultados ===
# Podemos obter o nome da melhor pipeline:
result.best_pipeline_name

'etrees+bert'

In [9]:
# Também podemos listar o valor das melhores métricas:
print(json.dumps({k: v.tolist() for k, v in result.best_metrics.items()},
                 indent=2,
                 ensure_ascii=False))

{
  "Weighted Precision": 0.6654836535453796,
  "Weighted Recall": 0.6892778873443604,
  "Weighted F1-score": 0.6349939107894897,
  "Kappa": 0.35971346497535706,
  "Neighbor Kappa": 0.35971346497535706
}


In [21]:
# Também podemos listar o valor de todas as métricas calculadas:
print(json.dumps({k: {k_: v_.tolist() for k_, v_ in v.items()} 
                  for k, v in result.metrics_history.items()},
                 indent=2,
                 ensure_ascii=False))

{
  "svm+features": {
    "Weighted Precision": 0.33119142055511475,
    "Weighted Recall": 0.5754923224449158,
    "Weighted F1-score": 0.42042914032936096,
    "Kappa": 0.0,
    "Neighbor Kappa": 0.0
  },
  "etrees+features": {
    "Weighted Precision": 0.4847065210342407,
    "Weighted Recall": 0.5809627771377563,
    "Weighted F1-score": 0.4975183606147766,
    "Kappa": 0.08787188678979874,
    "Neighbor Kappa": 0.08787188678979874
  },
  "lgbm+features": {
    "Weighted Precision": 0.49731308221817017,
    "Weighted Recall": 0.5689277648925781,
    "Weighted F1-score": 0.5195298194885254,
    "Kappa": 0.12839876115322113,
    "Neighbor Kappa": 0.12839876115322113
  },
  "xgb+features": {
    "Weighted Precision": 0.4791104793548584,
    "Weighted Recall": 0.550328254699707,
    "Weighted F1-score": 0.500983476638794,
    "Kappa": 0.09217804670333862,
    "Neighbor Kappa": 0.09217804670333862
  },
  "xgb+tfidf": {
    "Weighted Precision": 0.6343432664871216,
    "Weighted Recall":

In [19]:
# Por último, também é possível recuperar um DataFrame com as características
#   extraídas para os textos do dataset.
result.extras.df_features.head(3)

,text,adapted_dalechall,adverbs_before_main_verb_ratio,brunet_indice,clauses_per_sentence,coord_conj_ratio,coordinate_conjunctions_per_clauses,dialog_pron_ratio,easy_conj_ratio,flesch_indice,...,sentences_with_5_clauses,sentences_with_6_clauses,sentences_with_7_clauses,short_sentence_ratio,simple_word_ratio,std_noun_phrase,subord_conj_ratio,token_var_idx,verb_regency_score,words_before_main_verb
0,Apropriação cultural significa uma pessoa pode...,4.680675,0.352941,10.521848,2.428571,0.500000,0.235294,0.333333,0.237762,61.266434,...,0,0,0,0.285714,0.0,0.641128,0.500000,58.289064,1.0,2
1,A desigualdade social é um problema que afeta ...,4.744804,0.115385,11.063507,2.363636,0.600000,0.346154,0.000000,0.296610,35.633213,...,2,0,0,0.000000,0.0,0.916888,0.400000,63.981432,0.5,5
2,Hoje dependemos igualmente da qualidade de ens...,4.831121,0.900000,10.022355,2.500000,0.666667,0.400000,0.000000,0.273684,39.499276,...,0,0,0,0.000000,0.0,0.749943,0.333333,50.838519,1.0,1
